In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction

This note shows how to build benchmark with MIMIC-III data.  For more informations, please refer to github repo https://github.com/YerevaNN/mimic3-benchmarks.git.  There will be few codes in this note. 


## Environment prepare

Suggested to use Miniconda to create environment. 



In [2]:
conda create -n "mimic3" python=3.7.13

Retrieving notices: ...working... done
Channels:
 - rapidsai
 - nvidia
 - nodefaults
 - conda-forge
Platform: linux-64
Solving environment: \ | failed

PackagesNotFoundError: The following packages are not available from current channels:

  - python=3.7.13*

Current channels:

  - https://conda.anaconda.org/rapidsai
  - https://conda.anaconda.org/nvidia
  - https://conda.anaconda.org/nodefaults
  - https://conda.anaconda.org/conda-forge

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


Question:  Is it really need to use pyhon 3.7?  The following errors are caused by lib version controls

In [3]:
pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.


Here are the requirements.txt

Keras==2.1.2
tensorflow==1.15.0
protobuf==3.19.0
pandas==1.2.5
scikit-learn==1.0.2
numpy==1.16.5
pyyaml==6.0
tqdm

Expected errors:

ERROR: Could not find a version that satisfies the requirement tensorflow==1.15.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0)
ERROR: No matching distribution found for tensorflow==1.15.0

## Get MIMIC-III data

There are no instructions in github repo for getting MIMIC-III data. Assume this canbe downloaded from https://mimic.mit.edu/docs/iii/

## Benchmarking data

Since there are issues of pip install, the following code can not be verified. 

1. The following command takes MIMIC-III CSVs, generates one directory per `SUBJECT_ID` and writes ICU stay information to `data/{SUBJECT_ID}/stays.csv`, diagnoses to `data/{SUBJECT_ID}/diagnoses.csv`, and events to `data/{SUBJECT_ID}/events.csv`. This step might take around an hour.
```bash
       python -m mimic3benchmark.scripts.extract_subjects {PATH TO MIMIC-III CSVs} data/root/
```

2. The following command attempts to fix some issues (ICU stay ID is missing) and removes the events that have missing information. About 80% of events remain after removing all suspicious rows (more information can be found in [`mimic3benchmark/scripts/more_on_validating_events.md`](mimic3benchmark/scripts/more_on_validating_events.md)).
```bash
       python -m mimic3benchmark.scripts.validate_events data/root/
```

3. The next command breaks up per-subject data into separate episodes (pertaining to ICU stays). Time series of events are stored in ```{SUBJECT_ID}/episode{#}_timeseries.csv``` (where # counts distinct episodes) while episode-level information (patient age, gender, ethnicity, height, weight) and outcomes (mortality, length of stay, diagnoses) are stores in ```{SUBJECT_ID}/episode{#}.csv```. This script requires two files, one that maps event ITEMIDs to clinical variables and another that defines valid ranges for clinical variables (for detecting outliers, etc.). **Outlier detection is disabled in the current version**.
```bash
       python -m mimic3benchmark.scripts.extract_episodes_from_subjects data/root/
```

4. The next command splits the whole dataset into training and testing sets. Note that the train/test split is the same of all tasks.
```bash
       python -m mimic3benchmark.scripts.split_train_and_test data/root/
```

5. The following commands will generate task-specific datasets, which can later be used in models. These commands are independent, if you are going to work only on one benchmark task, you can run only the corresponding command.
```bash
       python -m mimic3benchmark.scripts.create_in_hospital_mortality data/root/ data/in-hospital-mortality/
       python -m mimic3benchmark.scripts.create_decompensation data/root/ data/decompensation/
       python -m mimic3benchmark.scripts.create_length_of_stay data/root/ data/length-of-stay/
       python -m mimic3benchmark.scripts.create_phenotyping data/root/ data/phenotyping/
       python -m mimic3benchmark.scripts.create_multitask data/root/ data/multitask/
```
